In [1]:
# --- Import Packages ---
import os
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path
from PIL import Image

In [ ]:
# ---- FOLDER CONFIG ----
project_root = r"D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE"
print(project_root)

D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE


In [3]:
VIDEOS_DIR = os.path.join(project_root, 'dataset', 'videos')
CLIPS_DIR = os.path.join(project_root, 'outputs', 'clips')
MODELS_DIR = os.path.join(project_root, 'outputs', 'models')
CROPPED_DIR = os.path.join(project_root, 'outputs', 'cropped_regions')
ANNOTATION_DIR = os.path.join(project_root, 'dataset', 'annotations', 'task2_labels')
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(CROPPED_DIR, exist_ok=True)
for sub in ['eye_left', 'eye_right', 'chin']:
    os.makedirs(os.path.join(CROPPED_DIR, sub), exist_ok=True)

In [4]:
# ---- CLASSES & MODEL CONFIG ----
CLASSES = ['eye_left', 'eye_right', 'chin']
YOLO_MODEL = 'yolov8n.pt'   # Pre-trained checkpoint
YOLO_PROJECT_NAME = 'yolov8_eyes_chin'
YOLO_EPOCHS = 50
YOLO_IMGSIZE = 640

In [5]:
# ---- STEP 1: EXTRACT FRAMES FOR MANUAL ANNOTATION ----
def extract_sample_frames(clips_dir, num_frames_per_clip=50, output_dir=None):
    if output_dir is None:
        output_dir = os.path.join(project_root, 'dataset', 'frames_to_annotate')
    os.makedirs(output_dir, exist_ok=True)
    clips = []
    for sub in ['eyes_positive', 'chin_positive', 'negative']:
        clips += list(Path(os.path.join(clips_dir, sub)).glob('*.mp4'))
    # Limit for manual annotation scope
    for clip_path in clips[:20]:  # Change 20 for more/less frames
        cap = cv2.VideoCapture(str(clip_path))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.isOpened() else 0
        if total_frames < num_frames_per_clip:
            step = 1
        else:
            step = max(1, total_frames // num_frames_per_clip)
        for frame_idx in range(0, total_frames, step):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            if ret:
                img_name = f"{clip_path.stem}_frame{frame_idx}.jpg"
                Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).save(os.path.join(output_dir, img_name))
        cap.release()
    print(f"Sample frames extracted to {output_dir}. Annotate them for eyes/chin using a tool (LabelImg/Roboflow/CVAT).")

In [22]:
extract_sample_frames(CLIPS_DIR)

Sample frames extracted to D:\5TH SEM\DL\DL PROJECT\DL_Project_Equine Pain\EQUINE PAIN CODE\dataset\frames_to_annotate. Annotate them for eyes/chin using a tool (LabelImg/Roboflow/CVAT).
